# Разрешение неоднозначности: WSD


In [161]:
import re
from collections import Counter
import os
from pprint import pprint
from pymystem3 import Mystem

In [162]:
m = Mystem()

Ниже происходит загрузка стоп-слов. Стоп слова были собраны из списка стоп-слов из примера +  к ним добавлены стоп-слова с сайта stopslov.net

In [163]:
def load_stop_words(stop_filename):
	''' загрузить список стоп-слов из файла, одно слово на строке '''
	with open(stop_filename, encoding = 'utf-8') as f:
		stopwords = [w.strip() for w in f.readlines() if w.strip()]
	return set(stopwords)

stopw = "stoplist_russian.txt"
print(load_stop_words(stopw))


{'ну', 'год', 'прочий', 'так что', 'обеспечить', 'и', 'Взыскательный', 'в ходе', 'эм', 'около', 'начать', 'нечто', 'реально', 'определенно', 'в настоящем времени', 'в лучших традициях', 'к примеру', 'очень', 'в', 'с учетом', 'примерно', 'долженствовать', 'нередко', 'путь', 'а также', 'бытует', 'никто', 'таков', 'и другие', 'осуществлять', 'длительный', 'знаковый', 'по моему мнению', 'так называемый', 'имею', 'лишь', 'ли', 'разнообразный', 'по-хорошему', 'и так далее', 'ибо', 'интересный', 'когда', 'реальный', 'в очередной раз', 'бытовать', 'крайне', 'инновационный', 'некто', 'производим работы', 'без сомнения', 'не секрет', 'со', 'значит', 'небывалый', 'в адрес', 'несколько', 'в условиях', 'без пяти минут', 'всем нам', 'но и', 'вновь', 'на', 'издревле', 'в целом', 'ещё', 'данный', 'ссылка', 'она', 'демонстрировать', 'абсолютно', 'деятельность', 'словно', 'без исключения', 'представленный', 'производить', 'ежели', 'исстари', 'и тому подобное', 'имеется', 'самый', 'всякий', 'с целью', 'ч

2. Препроцессинг
Перед тем, как из множества контекстов получить набор признаков, нужно 
(1) убрать лишние символы
(2) убрать стоп-слова
(3) лемматизировать тексты

In [164]:
def preprocessing(raw_text):
    clean_text = re.sub('\W+', ' ', raw_text) # \W = [^a-zA-Z0-9_]
    return clean_text

def lemmatize(input):
    return [lemma.strip() for lemma in m.lemmatize(preprocessing(input.lower())) if lemma.strip()]

text = "с виду воду, которая бьёт  рядом с французским городом Авен"
#m.lemmatize(text)

#r_text = "Он еле дождался, пока я доковыряюсь ключом в замке и, не снимая сапожек, прямиком пошёл в свою спальню."
#print(preprocessing(text))
print(lemmatize(text))

['с', 'вид', 'вода', 'который', 'бить', 'рядом', 'с', 'французский', 'город', 'авен']


In [165]:
def remove_stop_words(lemmas, stopwords):
    return ' '.join([word for word in lemmas if word not in stopwords])

#stop_words_filename = 'D:/GooDrive/0_2018_Edu_CL2/0_2018_Edu_HSE_Bak_CL2/WSD/WSD_Classifier/stoplist_russian.txt' 	# одно слово на строке
stop_words = load_stop_words(stopw)
raw_texts = ['Он', 'еле', 'дождался', 'пока', 'я', 'доковыряюсь', 'ключом', 'в', 'замке', 'и', 'не', 'снимая']
print(remove_stop_words(raw_texts, stop_words))

Он еле дождался доковыряюсь ключом замке снимая


# 1. Загрузка и адаптация корпуса
В качестве корпуса были взяты контексты для слова "лист". Датасет был собран с НКРЯ первым методом. Брался лист в двух значениях первое значение обозначенное 0 - бумажный/фанерный/связанный с производством лист. Второе значение обозначенное 1 - лист растения/дерева/связанный с природой. 


In [166]:
# библиотека для поддержки различных кодировок при чтении/записи файлов
# вообще, все import лучше хранить в начале файла, но в IPython'е это не всегда удобно
import codecs

def load_files(filename):
    # список, где будут накапливаться результаты
    results = []
    
    # открываем файл как переменную inp_file.
    # Важно помнить, что в этой переменной хранится не сам текстовый файл, а своего рода указатель на него.
    # Файл еще предстоит прочитать.
    # Конструкция with следит за тем, чтобы переменная, которая после as, существовала только внутри блока.
    # Благодаря этому файл закроется сразу же после использования
    with(codecs.open(filename, encoding = 'utf-16')) as inp_file:
        # читаем первую строку "в никуда" (не сохраняем результат).
        # Первая строка у нас — это названия колонок, они нам не нужны
        inp_file.readline()
        # Для каждой строки в файле
        for line in inp_file:
            # откусываем концы строк
            line = line.strip('\r\n')
            # строка.count(подстрока) возвращает количество вхождений подстроки в строку
            if line.count('\t') != 1:
            # специальное ключевое слово для перехода к следующей итерации цикла
                continue
            # делим строку по табуляции и складываем в каждую из двух переменных очередное значение
            sence, context = line.split('\t')
            # добавляем кортеж (sence, context) в общий список результатов
            cx = lemmatize(context)
            cx = remove_stop_words(cx, stop_words)
            results.append((int(sence), cx))
    # возвращаем результаты в виде списка кортежей (sence, context)
    return results

In [167]:
# указываем абсолютный или относительный путь
data = load_files("listalll.txt")

In [168]:
# напечатаем первые 10 элементов нашего корпуса
for item in data[:10]:
    print (item[0], item[1])

0 под угол 75 к поверхность аккуратно торопиться вырезать заштриховывать
0 открывать ящик бумага который большущий буква
0 к ветка рис 5 крупный из соломка фон белый
0 минута внимательно изучать полтора набирать компьютер текст
0 канцелярский принадлежность 1 карандаш 2 писчий бумага 1 папка причем
0 это место утрата накладывать тонкий белый бумага
0 из панцирь черепаха наклеивать фанера толщина 3 мм
0 черноликий подсолнечник архангельский крыло золотой нимб лепесток как
0 нержин покоряться взять бумага миг задумываться затем
0 разварка тленный мелкий рыбешка из под черный капуста выплевывать косточка


In [169]:
with codecs.open('lemma.txt', mode='w', encoding='utf-8') as wr:
    wr.write('sense\tcontext(lemmatized)\r\n')
    for item in data:
        wr.write(str(item[0]) + '\t' + item[1] + "\r\n")

В качестве классификаторов берем Байесовский MultinomialNB, K-соседей KneighborsClassifier и LinearSVC 
Для векторизации берем, как в примере, CountVectorizer, TfidfVectorizer

In [170]:
# математика
import numpy

# разные классификаторы
from sklearn.naive_bayes import MultinomialNB
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier

# векторизация
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# деление корпуса train/test
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

# Pipeline
from sklearn.pipeline import Pipeline

In [171]:
x_context = [item[1] for item in data]
y_labels = [item[0] for item in data]
print(x_context[:5])
print(y_labels[1])

['под угол 75 к поверхность аккуратно торопиться вырезать заштриховывать', 'открывать ящик бумага который большущий буква', 'к ветка рис 5 крупный из соломка фон белый', 'минута внимательно изучать полтора набирать компьютер текст', 'канцелярский принадлежность 1 карандаш 2 писчий бумага 1 папка причем']
0


In [172]:
def tokenize(input):
    return input.split(' ')

In [173]:
classifiers = [('SVM', LinearSVC), ('Bayes', MultinomialNB), ('KNeighbors', KNeighborsClassifier)]
vectorizers = [('Counts', CountVectorizer), ('TfIdf', TfidfVectorizer)]

In [174]:
def score_classifier(clf, metric='f1'):
    scores = cross_val_score(pipeline, numpy.asarray(x_context), numpy.asarray(y_labels), cv=5, scoring=metric)
    score = sum(scores) / len(scores)
    
    return score

In [175]:
for clf in classifiers:
    for vctr in vectorizers:
        pipeline = Pipeline([
    ('vectorizer', vctr[1]()),
    ('classifier', clf[1]()) ])
        print (clf[0], vctr[0], score_classifier(pipeline))

SVM Counts 0.667259224565
SVM TfIdf 0.684446312134
Bayes Counts 0.70120286403
Bayes TfIdf 0.687022200837
KNeighbors Counts 0.460369763902
KNeighbors TfIdf 0.638613911613


Самое высокое значение выдает сочетание СountVectorizer и классификатора MultinomialNB

In [176]:
def score_classifier(clf, metric='f1'):
    scores = cross_val_score(pipeline, numpy.asarray(x_tweets), numpy.asarray(y_labels), cv=5, scoring=metric)
    score = sum(scores) / len(scores)
    
    return score

In [177]:
scores = cross_val_score(pipeline, numpy.asarray(x_context), numpy.asarray(y_labels), cv=5, scoring='f1')
score = sum(scores) / len(scores)

print (score)
print (scores)

0.638613911613
[ 0.62222222  0.70714286  0.62121212  0.57831325  0.6641791 ]


In [178]:
#import sklearn

from nltk.corpus import stopwords
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold
from sklearn.naive_bayes import MultinomialNB
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.model_selection import train_test_split

import numpy as np

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import os, sys, codecs

In [179]:
texts = [item[1] for item in data]
for i in range(10):
    print (texts[i])
    
len(texts)

print (3974 *16344 * 4)

под угол 75 к поверхность аккуратно торопиться вырезать заштриховывать
открывать ящик бумага который большущий буква
к ветка рис 5 крупный из соломка фон белый
минута внимательно изучать полтора набирать компьютер текст
канцелярский принадлежность 1 карандаш 2 писчий бумага 1 папка причем
это место утрата накладывать тонкий белый бумага
из панцирь черепаха наклеивать фанера толщина 3 мм
черноликий подсолнечник архангельский крыло золотой нимб лепесток как
нержин покоряться взять бумага миг задумываться затем
разварка тленный мелкий рыбешка из под черный капуста выплевывать косточка
259804224


In [180]:
vectorizer = TfidfVectorizer()

texts = [item[1] for item in data]
vectorized = vectorizer.fit_transform(texts)
vectorized

print (vectorized[0], vectorized[0].shape)

  (0, 2303)	0.241507368822
  (0, 3526)	0.353295818119
  (0, 40)	0.413590017053
  (0, 2290)	0.33104302326
  (0, 99)	0.320029785052
  (0, 3453)	0.391337222194
  (0, 554)	0.33104302326
  (0, 1007)	0.413590017053 (1, 3879)


In [182]:
clf = RandomForestClassifier(n_estimators=300)
y_labels = [item[0] for item in data]
clf.fit(vectorized.toarray(), y_labels)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [183]:
clf.predict(vectorized[0].toarray())
print (y_labels[0])

0


In [184]:
y_labels = [item[0] for item in data]
X_train, X_test, y_train, y_test = train_test_split(vectorized, y_labels, test_size=0.1, random_state=42)
len(y_labels)

1378

In [185]:
names = vectorizer.get_feature_names()

In [186]:
items = clf.predict(X_test.toarray())
right = 0
for i in range(len(items)):
    if items[i] == y_test[i]:
        right += 1
        
print ('Total: %d\nRight: %d\nScore: %.3f' % (len(items), right, float(right) / len(items)))

words = {}
for i in range(len(clf.feature_importances_)):
    words[names[i]] = clf.feature_importances_[i]
    
for item in sorted(words, key = lambda word: words[word], reverse=True):
    print (item, words[item])

Total: 138
Right: 138
Score: 1.000
бумага 0.0500685329764
дерево 0.01317388195
растение 0.00917276121276
из 0.00913247579959
который 0.00900946510602
стол 0.00818337320443
чистый 0.00748741389809
желтый 0.00722432442046
написать 0.0065274443612
взять 0.00618710705904
цветок 0.00602937583749
быть 0.0060228253982
они 0.00584155823346
опадать 0.00566700773981
стебель 0.00554776069962
как 0.00551358582673
от 0.00517653030286
лист 0.00497471902884
за 0.00494630480692
вода 0.00493310296393
зеленый 0.00490767835848
писать 0.00481012699319
себя 0.00469766994222
белый 0.00460060917863
книга 0.00437051527409
то 0.00418354902013
исписывать 0.00398585801666
один 0.00395865125762
корень 0.00393875743846
ветер 0.00383727229716
лес 0.00383134216722
дождь 0.00381893414212
большой 0.00377616011126
или 0.00376011724771
но 0.00374757551791
под 0.00349847258717
дело 0.00347795419008
газетный 0.00343583714532
весь 0.00330706743507
бы 0.00324298088285
ветка 0.00318856522268
это 0.00315114887717
клен 0.00302

заводить 0.000205953427994
приставать 0.00020521655267
батиметрический 0.000205205434533
плоскость 0.000205171041075
потолок 0.00020508439762
вглядываться 0.00020504662689
занимать 0.000204494244115
литой 0.000204473025347
лента 0.000204244190752
ладонь 0.000203958192301
наступать 0.000203947827207
значиться 0.000203754771976
вздутый 0.000203400236194
немецкий 0.000203099111892
груда 0.00020272135735
наугад 0.00020256652291
мадам 0.000202420954787
графический 0.000202393081265
убирать 0.000201317329314
засиживать 0.000201194450019
задумчиво 0.000201165893524
еле 0.000201051275873
картонный 0.00020087936243
акварельный 0.000200418495582
море 0.000200356854705
жар 0.000200246759386
вслед 0.000200033754989
мелодичный 0.000199226356576
дописывать 0.000198942320758
жухнуть 0.000198842282447
рожь 0.000198725889202
оседать 0.000198328897337
губа 0.000198130064251
полированный 0.000197864379743
зал 0.000197720894346
кружево 0.000197509645119
путаться 0.000197355584474
прозрачно 0.0001973023542

рост 8.42603990238e-05
чертополох 8.40818526034e-05
изюм 8.4065818205e-05
труд 8.40275089279e-05
варить 8.39934322823e-05
название 8.39784590085e-05
обозначаться 8.39629696593e-05
прочь 8.39181749036e-05
немощеный 8.38387946444e-05
платформа 8.38234479838e-05
вагнер 8.36766824899e-05
расправляться 8.35644687958e-05
22x36 8.35296680642e-05
озабоченно 8.34730215713e-05
опасный 8.34227299422e-05
окладной 8.34094271952e-05
базилика 8.3392415879e-05
1916 8.33458128182e-05
веерный 8.32836447137e-05
поклевывать 8.31626033552e-05
увеличиваться 8.31241022838e-05
преждевременный 8.30205388825e-05
задумчивый 8.29463350586e-05
притом 8.29082439676e-05
акантовый 8.26777678572e-05
фалунский 8.26293444057e-05
кусочек 8.25728029213e-05
безлистный 8.23027092514e-05
углекислый 8.22924327084e-05
треск 8.22464428631e-05
кланяться 8.21586147648e-05
поросль 8.21491908451e-05
обратный 8.2102372592e-05
размышлять 8.19943651957e-05
невинно 8.19688645915e-05
достигать 8.1862179976e-05
пятьдесят 8.18596519336e-0

отделяться 3.8913927295e-05
лениво 3.88892018875e-05
небрежно 3.88845991758e-05
насорить 3.87402562824e-05
смущать 3.87381979687e-05
принтер 3.86776745801e-05
подошва 3.86693879492e-05
инородный 3.86139139566e-05
наборный 3.85490376739e-05
перевал 3.82805575161e-05
играть 3.82380739492e-05
14 3.82051926984e-05
сметать 3.81851931468e-05
пурпурный 3.81580968065e-05
раструб 3.81422266589e-05
прибрежный 3.81183115791e-05
кружить 3.80112487801e-05
шов 3.7939687875e-05
понимать 3.79371860441e-05
устраивать 3.78986909983e-05
отделение 3.78873801379e-05
подстригать 3.78384688633e-05
саван 3.78331842103e-05
битва 3.78190922044e-05
продажа 3.76955894552e-05
бездна 3.75798888666e-05
обрыв 3.75652104354e-05
взяться 3.75374805439e-05
участвовать 3.73729689911e-05
корешок 3.73239234553e-05
прикасаться 3.72176616303e-05
вощев 3.72173953072e-05
бисерный 3.7176841811e-05
лишать 3.71595168466e-05
пюпитр 3.71266608861e-05
латунный 3.71264096751e-05
пугливо 3.71243708016e-05
лавров 3.70937687855e-05
зрите

устройство 5.11738686447e-06
автобиография 5.08651066882e-06
обременительный 5.05678824184e-06
ужасно 5.05514506522e-06
раздвигать 4.90051430215e-06
давидович 4.8207585826e-06
всяческий 4.44022067377e-06
подснежник 4.3945993651e-06
застилать 4.37768747676e-06
малицкий 4.37525530867e-06
оберточный 4.06257236186e-06
зачеркивать 4.06230669519e-06
потрясать 4.01888197363e-06
пишущий 4.00564338155e-06
пропадать 3.90853212716e-06
повесть 3.83608571041e-06
несходный 3.77292459725e-06
ефимович 3.76051717034e-06
упражнение 3.75363709101e-06
пирожное 3.70730988455e-06
жара 3.6557512105e-06
филипп 3.64297254603e-06
скромный 3.60122043505e-06
сборник 3.57987597407e-06
облако 3.57698049696e-06
исследовать 3.54539820162e-06
грамматический 3.52647269549e-06
испачкать 3.50208385897e-06
тумбочка 3.44355924149e-06
аленкина 3.42822239056e-06
скомкивать 3.40148426498e-06
этаж 3.38586898924e-06
сдерживать 3.38077208432e-06
монограмма 3.13028836911e-06
бланк 2.95766766414e-06
генштаб 2.94259250455e-06
крупн

In [187]:
#print data[10][0], data[10][1]
#items = clf.predict(vectorized[10])
#print items[0]
#print X_test
items = clf.predict(X_test.toarray())
right = 0
for i in range(len(items)):
    if items[i] == y_test[i]:
        right += 1
    #print items[i], y_test[i]
names = vectorizer.get_feature_names()
words = {}
print ('Total: %d\nRight: %d\nScore: %.3f' % (len(items), right, float(right) / len(items)))
for i in range(len(clf.feature_importances_)):
    words[names[i]] = clf.feature_importances_[i]
    
for item in sorted(words, key = lambda word: words[word], reverse=True):
    print (item, words[item])

Total: 138
Right: 138
Score: 1.000
бумага 0.0500685329764
дерево 0.01317388195
растение 0.00917276121276
из 0.00913247579959
который 0.00900946510602
стол 0.00818337320443
чистый 0.00748741389809
желтый 0.00722432442046
написать 0.0065274443612
взять 0.00618710705904
цветок 0.00602937583749
быть 0.0060228253982
они 0.00584155823346
опадать 0.00566700773981
стебель 0.00554776069962
как 0.00551358582673
от 0.00517653030286
лист 0.00497471902884
за 0.00494630480692
вода 0.00493310296393
зеленый 0.00490767835848
писать 0.00481012699319
себя 0.00469766994222
белый 0.00460060917863
книга 0.00437051527409
то 0.00418354902013
исписывать 0.00398585801666
один 0.00395865125762
корень 0.00393875743846
ветер 0.00383727229716
лес 0.00383134216722
дождь 0.00381893414212
большой 0.00377616011126
или 0.00376011724771
но 0.00374757551791
под 0.00349847258717
дело 0.00347795419008
газетный 0.00343583714532
весь 0.00330706743507
бы 0.00324298088285
ветка 0.00318856522268
это 0.00315114887717
клен 0.00302

подсолнух 0.000215228043371
кипяток 0.000214814290483
ласковый 0.000214529055887
ножевидный 0.000214501103624
прорисовывать 0.000213995670682
заяц 0.000213803718079
изящный 0.000213641454166
падение 0.000213134878425
удаление 0.000212636839512
ива 0.000212253615187
кокосовый 0.000212160159341
ароматный 0.000211991824842
печь 0.00021145400008
инструмент 0.000210965457499
предводитель 0.000210545874851
затея 0.000210262140262
ломаный 0.000210122898586
нибудь 0.000209795702127
хлорофилл 0.000209671924571
горьковатый 0.000208910741373
бескровный 0.000208834371784
жила 0.000208662071641
вариант 0.000208623499158
бред 0.000208545963931
прилипать 0.000208190822106
сверкать 0.000208016405938
поглощать 0.000207500643576
синдикат 0.000207380740213
отсутствие 0.00020708122073
любить 0.000207077594936
махать 0.000206873599688
кувшинка 0.000206563725443
мешок 0.000206196748815
предшествовать 0.000206042226665
заводить 0.000205953427994
приставать 0.00020521655267
батиметрический 0.000205205434533
п

закладывать 0.000117166802492
обвертывать 0.000117165176309
кабинет 0.000117132211907
разветвление 0.000117124283078
видимо 0.000117044596899
взгляд 0.000116830439229
притягивать 0.00011677674982
начинить 0.000116750487257
седина 0.000116691316992
подросток 0.000116622088452
портмоне 0.00011656418124
профиль 0.000116474149996
значительный 0.000116446593074
железо 0.000116442265458
покачиваться 0.000116341356624
высокий 0.000116322263697
складочка 0.000116189256685
перрон 0.000116167261833
волос 0.000116036593536
нависать 0.000115789283886
фраза 0.000115779665839
выращивание 0.000115729362166
холодильник 0.000115448995437
правильный 0.000115396507028
дополнительный 0.000115377153142
досада 0.000115334683191
моряк 0.000115223150843
нимб 0.000115208705239
семья 0.000115208127309
щи 0.000115147229699
кудрявый 0.000115119390693
передо 0.0001149641348
лужа 0.000114903916824
обнаженный 0.000114880198
могучий 0.000114782496401
приложение 0.000114734823626
брезентовый 0.000114713089105
прихотли

солома 5.62724993293e-05
некротизация 5.60408544152e-05
отживать 5.59759368964e-05
внутрь 5.58928994692e-05
сталь 5.58637735745e-05
искривлять 5.5839196315e-05
цепочка 5.58067257419e-05
занимательный 5.55614220328e-05
отдаление 5.55328264503e-05
идти 5.54749216271e-05
золотеть 5.54687799794e-05
ниспадать 5.54323285619e-05
видать 5.54267327249e-05
стремиться 5.54160312755e-05
баглай 5.53223122983e-05
тайна 5.52114724922e-05
обыкновенный 5.51975961239e-05
переводиться 5.51302721945e-05
выкладывать 5.51039122636e-05
615 5.505889342e-05
ореховый 5.50318223896e-05
пристально 5.49356887917e-05
устремляться 5.49136876804e-05
блестящий 5.48477829075e-05
салатовый 5.48459529561e-05
35 5.48400022679e-05
изменять 5.48080320968e-05
выигрышный 5.46696668807e-05
град 5.46188423235e-05
залог 5.45991308146e-05
трилистник 5.45969116484e-05
канна 5.44823059391e-05
старушка 5.44447248971e-05
ржаветь 5.43977053741e-05
петрович 5.43469126641e-05
владелец 5.43432549693e-05
расходный 5.42965123651e-05
горох 

красоваться 3.41238858542e-05
повышение 3.41078201821e-05
полотно 3.40874186062e-05
печка 3.40427394884e-05
однажды 3.40098967732e-05
упругость 3.38164247501e-05
циновка 3.38025966228e-05
теменец 3.37801729089e-05
поперек 3.37217464337e-05
эстрагон 3.36123831727e-05
накладывать 3.3594929858e-05
посмотреть 3.35804625076e-05
рывок 3.35078422728e-05
отрастать 3.34992477142e-05
приближать 3.34478806061e-05
искалывать 3.34430037835e-05
золотить 3.34167762102e-05
чуять 3.33910505675e-05
вишенник 3.3386783002e-05
разгонистый 3.33275819517e-05
торопливый 3.32369096948e-05
передвечерний 3.32305548328e-05
двояко 3.31252778663e-05
андрей 3.30401820919e-05
множайший 3.29750376272e-05
выстрел 3.29718628813e-05
знаменитость 3.29078188818e-05
антон 3.28553485953e-05
мелованный 3.27947993432e-05
поддувать 3.27668527252e-05
почечный 3.26862375847e-05
пастбище 3.26511744016e-05
зашелестеть 3.25666215842e-05
преимущественно 3.25585438796e-05
разорять 3.24820969484e-05
истома 3.24009861777e-05
четверостиш

витька 2.48283301274e-07
планшетка 1.98341545329e-07
стаскивать 1.90203536228e-07
чей 1.80996811821e-07
канал 1.63922469491e-07
карточка 1.50700448956e-07
гроссбух 1.41732516201e-07
арифметический 1.22849703325e-07
диван 1.18116980044e-07
хадж 9.66838974893e-08
райком 8.60632754189e-08
отчетливый 7.48099682661e-08
глянуть 7.45418386288e-08
мельком 4.29794794029e-08
порыться 4.15524738899e-08
покоряться 2.91525666529e-08
роскошный 1.29278188651e-08
придавливать 1.21539128124e-08
уезд 1.16884760674e-08
донимать 6.92896022405e-09
примета 0.0
подходить 0.0
беспорядок 0.0
посвящать 0.0
изрезанный 0.0
378 0.0


In [148]:
names = vectorizer.get_feature_names()
len(names)

3916